In [14]:
# %%

import os
import sys
import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import openpyxl
import pandas as pd
import math
import itertools
import matplotlib.patches as patches
import seaborn as sns
from decimal import Decimal
import glob
import csv
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sklearn.svm 
import sklearn.metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers



arch="v100"

y_label = []

category = "mybench"
dataName = "%s/power-combine-feature-%s-%s.csv"%(arch, category, arch)
data0 = pd.read_csv(dataName, index_col=0 )
category = "risky"
dataName = "%s/power-combine-feature-%s-%s.csv"%(arch, category, arch)
data1 = pd.read_csv(dataName, index_col=0 )

for i in range(data0.shape[0]):
    y_label.append(0)
for i in range(data1.shape[0]):
    y_label.append(1)
print(data0.shape)
print(data1.shape)
X = pd.concat([data0, data1])
print("done")
#%%

(694, 3175)
(169, 3175)
done


In [15]:
#%%
def tfnnClassify(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    model = tf.keras.Sequential()
    
    #model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
   
    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

    model.fit(x=X_train,y=y_train,
            validation_split=0.25,
            epochs=50,
            batch_size = 256,
            class_weight={1:5, 0:1})

    loss,train_accuracy =  model.evaluate(x=X_train, y=y_train)
    loss,test_accuracy = model.evaluate(x=X_test, y=y_test)
    return train_accuracy,  test_accuracy

def svmClassify(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    clf = sklearn.svm.SVC(gamma=0.001, class_weight='balanced', C=1.0, probability=True, random_state=1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)      
    train_accuracy = sklearn.metrics.accuracy_score(y_train, y_pred)
    
    y_pred = clf.predict(X_test)
    probas = clf.predict_proba(X_test)    
    test_accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
    return train_accuracy,  test_accuracy
def knnClassify(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    model = tf.keras.Sequential()
    
    model.add(layers.Dense(1, kernel_initializer='normal', activation='sigmoid'))
  
    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

    model.fit(x=X_train,y=y_train,
            validation_split=0.25,
            epochs=50,
            batch_size = 256,
            class_weight={1:5, 0:1})

    loss,train_accuracy =  model.evaluate(x=X_train, y=y_train)
    loss,test_accuracy = model.evaluate(x=X_test, y=y_test)
    return train_accuracy,  test_accuracy

def select_features(X, y, topN):
    clf = sklearn.svm.SVC(kernel='linear')    
    clf.fit(X, y)
    coef_list = clf.coef_[0][:]
    coef_list =abs(coef_list)
    top_idx = (np.argsort(coef_list)[-topN:]).tolist()
    top_idx.reverse()
    return top_idx, clf.coef_[0][top_idx]
#%%

In [16]:
#%%
y = y_label

X = X.dropna(axis=1)
X = X.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
#for i in range(X.shape[1]):
#for i in range(X.shape[1]):
#    for j in X.values[:, i]:
#        if not np.isfinite(j):
#            print(X.columns[i])
#            print(j)
     
print(X.shape)
X_std =  StandardScaler().fit_transform(X.values)
print(X_std.shape[1])

topN = 100
top_idx, coef_list = select_features(X_std, y, topN)
print(coef_list[topN-1])




topN_idx = top_idx[:topN]
X_input = X_std[:, topN_idx]
train_accuracy, test_accuracy = tfnnClassify(X_input, y)
print("nn", train_accuracy, test_accuracy)
train_accuracy, test_accuracy = knnClassify(X_input, y)
print("knn",train_accuracy, test_accuracy)
train_accuracy, test_accuracy = svmClassify(X_input, y)
print("svm",train_accuracy, test_accuracy)
#%%

(863, 3155)
3155
-0.034940668196799626
Train on 485 samples, validate on 162 samples
Epoch 1/50
485/485 [==============================] - 0s 636us/sample - loss: 0.7851 - acc: 0.6784 - val_loss: 0.7392 - val_acc: 0.6914
Epoch 2/50
485/485 [==============================] - 0s 16us/sample - loss: 0.7121 - acc: 0.6948 - val_loss: 0.6897 - val_acc: 0.6852
Epoch 3/50
485/485 [==============================] - 0s 14us/sample - loss: 0.6524 - acc: 0.6928 - val_loss: 0.6488 - val_acc: 0.7160
Epoch 4/50
485/485 [==============================] - 0s 14us/sample - loss: 0.6046 - acc: 0.7237 - val_loss: 0.6150 - val_acc: 0.7160
Epoch 5/50
485/485 [==============================] - 0s 14us/sample - loss: 0.5656 - acc: 0.7402 - val_loss: 0.5866 - val_acc: 0.7346
Epoch 6/50
485/485 [==============================] - 0s 15us/sample - loss: 0.5346 - acc: 0.7567 - val_loss: 0.5617 - val_acc: 0.7531
Epoch 7/50
485/485 [==============================] - 0s 15us/sample - loss: 0.5047 - acc: 0.7526 - val_

Epoch 9/50
485/485 [==============================] - 0s 11us/sample - loss: 0.6466 - acc: 0.5608 - val_loss: 0.6776 - val_acc: 0.5556
Epoch 10/50
485/485 [==============================] - 0s 12us/sample - loss: 0.6392 - acc: 0.5670 - val_loss: 0.6695 - val_acc: 0.5679
Epoch 11/50
485/485 [==============================] - 0s 11us/sample - loss: 0.6279 - acc: 0.5814 - val_loss: 0.6616 - val_acc: 0.5617
Epoch 12/50
485/485 [==============================] - 0s 11us/sample - loss: 0.6197 - acc: 0.6021 - val_loss: 0.6541 - val_acc: 0.5617
Epoch 13/50
485/485 [==============================] - 0s 11us/sample - loss: 0.6114 - acc: 0.6144 - val_loss: 0.6468 - val_acc: 0.5679
Epoch 14/50
485/485 [==============================] - 0s 11us/sample - loss: 0.6028 - acc: 0.6289 - val_loss: 0.6399 - val_acc: 0.5741
Epoch 15/50
485/485 [==============================] - 0s 11us/sample - loss: 0.5953 - acc: 0.6289 - val_loss: 0.6333 - val_acc: 0.5864
Epoch 16/50
485/485 [============================